## **train셋과 동일하게 test셋도 전처리 과정을 거쳤습니다.**


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import requests
from bs4 import BeautifulSoup

import re

import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
#데이터셋 불러오기 
test_df = pd.read_csv('/content/drive/MyDrive/BDA 공모전/tmk_bda_test.csv', index_col = 0, encoding = 'utf-8')

#브랜드 사전 불러오기 
brand = pd.read_csv('/content/drive/MyDrive/BDA 공모전/brand_complete.csv')

#대분류 불러오기
big = pd.read_csv('/content/drive/MyDrive/BDA 공모전/새벽하스.../bigcategory.csv', index_col=0)

#중분류 불러오기 
mid = pd.read_csv('/content/drive/MyDrive/BDA 공모전/새벽하스.../middlecategory.csv', index_col=0)

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/BDA 공모전/BDA 공모전
#데이터셋 불러오기 
test_df = pd.read_csv('tmk_bda_test.csv', index_col = 0, encoding = 'utf-8')

#브랜드 사전 불러오기 
brand = pd.read_csv('brand_complete.csv')

%cd /content/drive/MyDrive/Colab Notebooks/BDA 공모전/BDA 공모전/새벽하스...
#대분류 불러오기
big = pd.read_csv('bigcategory.csv', index_col=0)

#중분류 불러오기 
mid = pd.read_csv('middlecategory.csv', index_col=0)

[Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/BDA 공모전/BDA 공모전'
/content


FileNotFoundError: ignored

# 기본 변수

In [ ]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19660 entries, 45875 to 65534
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   scd            19660 non-null  int64  
 1   product_name   19660 non-null  object 
 2   net_order_qty  19660 non-null  int64  
 3   net_order_amt  19660 non-null  float64
 4   gender         19660 non-null  object 
 5   age_grp        19660 non-null  int64  
 6   employee_yn    19660 non-null  object 
 7   order_date     19660 non-null  int64  
 8   prime_yn       0 non-null      float64
dtypes: float64(2), int64(4), object(3)
memory usage: 1.5+ MB


In [ ]:
######두번째 변수: 총 주문금액
  # 주문량 x 주문금액 후 > scd 별로 모두 더함 
test_df['total_order_amt'] = test_df['net_order_qty'] * test_df['net_order_amt']

total_amt_data = pd.DataFrame(test_df.groupby('scd').sum())['total_order_amt']

test_df.drop('total_order_amt', axis=1, inplace=True)
test_df = pd.merge(test_df, total_amt_data, on='scd')


######네번째 변수: 브랜드
#데이터프레임을 맵핑할 dictionary로 바꾸기 
bdic = dict(zip(brand['product_name'], brand['brand']))

#브랜드사전에 있는 제품은 그에 맞는 '브랜드'명을 저장 (없으면 nan)
test_df['brand'] = test_df['product_name'].map(bdic).fillna(np.nan)



######다섯 번째 변수: festival
test_df['festival'] = 0
test_df['order_date']= test_df['order_date'].astype(int)

# def func(x):
#  if x < 11:
#    return 1
#  else:
#    return 0

# #10일 기준으로 이진분류

for i in range(len(test_df)):
 if (str(test_df['order_date'][i])[-2:] in ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10']):
  test_df['festival'][i] = 1

######여섯번째 변수 : salary
test_df['salary'] = 0

for i in range(len(test_df)):
 if (test_df['employee_yn'][i] == 'N') and (str(test_df['order_date'][i])[-2:] in ['10', '11', '12', '25', '26', '27']):
   test_df['salary'][i] = 1

for i in range(len(test_df)):
  if (test_df['employee_yn'][i] == 'Y') and (str(test_df['order_date'][i])[-2:] in ['21', '22', '23']):
    test_df['salary'][i] = 1

In [ ]:
test_df['festival'].value_counts()

1    10322
0     9338
Name: festival, dtype: int64

In [ ]:
# ######첫번재 변수: 고객 정보

# # 10대, 2030F/M, 4050F/M, 60대의 6개 분류
for i in range(len(test_df)):
  if test_df.iloc[i,5] == 1:
    test_df.loc[i,'age_gen'] = '10 FM'
  elif (test_df.iloc[i,5] == 2) | (test_df.iloc[i,5] == 3):
    if test_df.iloc[i,4] == 'F':
      test_df.loc[i,'age_gen'] = '2030 F'
    else:
      test_df.loc[i,'age_gen'] = '2030 M'
  elif (test_df.iloc[i,5]== 4) | (test_df.iloc[i,5]==5):
    if test_df.iloc[i,4] == 'F':
      test_df.loc[i,'age_gen'] = '4050 F'
    else:
      test_df.loc[i,'age_gen'] = '4050 M'
  else:
    test_df.loc[i,'age_gen'] = '60 FM'

In [ ]:
test_df

,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,employee_yn,order_date,prime_yn,total_order_amt,brand,festival,salary,age_gen
0,20230101964282,비비고 스팸부대찌개 460g,1,8.161946,M,4,N,20230101,NaN,32.658027,비비고,1,0,4050 M
1,20230101964282,비비고 두부김치찌개 460g,1,7.920083,M,4,N,20230101,NaN,32.658027,비비고,1,0,4050 M
2,20230101964282,비비고 차돌된장찌개 460g,1,8.161946,M,4,N,20230101,NaN,32.658027,비비고,1,0,4050 M
3,20230101964282,비비고 수삼갈비탕 400g,1,8.414052,M,4,N,20230101,NaN,32.658027,비비고,1,0,4050 M
4,20230101970142,삼호 생선살어묵 야채 200g,1,8.098947,M,4,Y,20230101,NaN,52.732733,삼호,1,0,4050 M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19655,20230103990277,비비고 왕교자 455gx2개,1,8.917177,F,3,N,20230103,NaN,8.917177,비비고,1,0,2030 F
19656,20230103990367,[2023설사원선물신청]프레시웨이 LA갈비 선물세트 3.2kg,1,12.204807,M,3,Y,20230103,NaN,12.204807,타브랜드,1,0,2030 M
19657,20230103991606,비비고 왕교자 455gx2개,4,10.437903,M,3,Y,20230103,NaN,41.751611,비비고,1,0,2030 M
19658,20230103995966,비비고 왕교자 455gx2개,1,8.573574,F,3,N,20230103,NaN,8.573574,비비고,1,0,2030 F


In [ ]:
test_df[test_df['salary']==1]

,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,employee_yn,order_date,prime_yn,age_gen,total_order_amt,brand,salary
863,2.023013e+13,비비고 왕교자 1.05kg+비비고 수제진한고기만두400g*2ea,1.0,9.320091,F,4.0,N,20230127,NaN,4050 F,28.118057,비비고,1
864,2.023013e+13,[500개한정]비비고 진한김치만두400g*2X2개,1.0,9.407222,F,4.0,N,20230127,NaN,4050 F,28.118057,비비고,1
865,2.023013e+13,[500개한정] 고메 소보로 고르곤졸라 시카고 피자370gx2개,1.0,9.390743,F,4.0,N,20230127,NaN,4050 F,28.118057,고메,1
866,2.023013e+13,[100개한정]비비고 감자탕 460gX3개,1.0,9.386812,F,4.0,N,20230126,NaN,4050 F,65.616034,비비고,1
867,2.023013e+13,[1000개한정]비비고 왕교자 1.05kgX2개,1.0,9.461721,F,4.0,N,20230126,NaN,4050 F,65.616034,비비고,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19599,2.023013e+13,[500개한정]비비고 갈비탕 400gX3개,1.0,9.601166,F,4.0,N,20230126,NaN,4050 F,18.102636,비비고,1
19600,2.023013e+13,[500개한정]해찬들 고기찍어먹는장 찍장 쌈장 300gX2개,1.0,8.501470,F,4.0,N,20230126,NaN,4050 F,18.102636,해찬들,1
19601,2.023013e+13,The 더건강한 닭가슴살 통살 1000g(100g*10)(유통기한23.02.19까지),1.0,9.669283,F,3.0,N,20230126,NaN,2030 F,9.669283,더건강한,1
19602,2.023013e+13,[300개한정]비비고 잔칫집모둠잡채644gx2개,1.0,9.315511,F,5.0,N,20230126,NaN,4050 F,9.315511,비비고,1


# [] 변수 생성

In [ ]:
#product_name은 수정용. real_product은 원래용꺼. 
test_df['real_product'] = test_df['product_name']

In [ ]:
test_df['product_name'].value_counts()

고메 중화짬뽕 652g                          230
비비고 사골곰탕 500g                         180
비비고 물만두 370gx2개                       162
밀당의고수 바삭한 김말이 400g                    161
고메 바삭쫄깃한 탕수육 450g                     148
                                     ... 
투썸 에이리스트 스틱커피 카페라떼 10입                  1
백설 한식예찬 3kg                             1
[500개한정] 더건강한 닭가슴살 큐브톡톡 ALL 96gx2개      1
동치미물냉면 4인분                              1
햇반 솥반 꿀약밥 210gX18개(1box)                1
Name: product_name, Length: 2482, dtype: int64

In [ ]:
######한정, 선할인, 콜라보, 설날, 새벽배송 등등등 

#[이벤트] 들고오기 
event_list = []
product_list = test_df['product_name'].unique()

for product in product_list:

  start_idx = product.find('[')
  end_idx = product.find(']')

  # data.loc[data['product_name'] == product, 'product_name'] == re.search(r'\[(.*?)\]', product).group(1)
  
  if start_idx != -1 and end_idx != -1:
      extracted_value = product[start_idx:end_idx+1]
      event_list.append(extracted_value)

event_list = list(set(event_list))

In [ ]:
###변수 생성
test_df['limit'] = 0
test_df['pre_dcnt'] = 0
test_df['colab'] = 0
test_df['new_years'] = 0
test_df['early_del'] = 0
test_df['exclusive'] = 0
test_df['dcnt'] = 0
test_df['box'] = 0
test_df['mid_cat2'] = 0

In [ ]:
# 00개 한정 

for i, product_name in enumerate(test_df['product_name']):

  start_idx = product_name.find('[')
  end_idx = product_name.find(']')

  if start_idx != -1 and end_idx != -1:
      value = product_name[start_idx:end_idx+1]

      if '한정' in value:
        test_df['limit'][i] = 1
        new_product_name = product_name.replace(value, '')
        test_df['product_name'][i] = new_product_name

In [ ]:
# 선할인

for i, product_name in enumerate(test_df['product_name']):

  start_idx = product_name.find('[')
  end_idx = product_name.find(']')

  if start_idx != -1 and end_idx != -1:
      value = product_name[start_idx:end_idx+1]

      if '선할인' in value:
        test_df['pre_dcnt'][i] = 1
        new_product_name = product_name.replace(value, '')
        test_df['product_name'][i] = new_product_name

In [ ]:
# 콜라보

for i, product_name in enumerate(test_df['product_name']):

  start_idx = product_name.find('[')
  end_idx = product_name.find(']')

  if start_idx != -1 and end_idx != -1:
      value = product_name[start_idx:end_idx+1]

      if '침착맨' in value:
        test_df['colab'][i] = 1
        new_product_name = product_name.replace(value, '')
        test_df['product_name'][i] = new_product_name

In [ ]:
#설날 세트, 특가, 캠페인 등 설날 관련 상품

for i, product_name in enumerate(test_df['product_name']):

  start_idx = product_name.find('[')
  end_idx = product_name.find(']')

  if start_idx != -1 and end_idx != -1:
      value = product_name[start_idx:end_idx+1]

      if '23설' in value:
        test_df['new_years'][i] = 1
        new_product_name = product_name.replace(value, '')
        test_df['product_name'][i] = new_product_name

      if 'CJ공식몰_23설선물세트특가' in value:
        test_df['new_years'][i] = 1
        new_product_name = product_name.replace(value, '')
        test_df['product_name'][i] = new_product_name

In [ ]:
# 새벽배송

for i, product_name in enumerate(test_df['product_name']):

  start_idx = product_name.find('[')
  end_idx = product_name.find(']')

  if start_idx != -1 and end_idx != -1:
      value = product_name[start_idx:end_idx+1]

      if '새벽배송' in value:
        test_df['early_del'][i] = 1
        new_product_name = product_name.replace(value, '')
        test_df['product_name'][i] = new_product_name

In [ ]:
# CJ더마켓ONLY 상품

for i, product_name in enumerate(test_df['product_name']):

  start_idx = product_name.find('[')
  end_idx = product_name.find(']')

  if start_idx != -1 and end_idx != -1:
      value = product_name[start_idx:end_idx+1]

      if 'ONLY' in value:
        test_df['exclusive'][i] = 1
        new_product_name = product_name.replace(value, '')
        test_df['product_name'][i] = new_product_name

In [ ]:
# 앱전용특가

for i, product_name in enumerate(test_df['product_name']):

  start_idx = product_name.find('[')
  end_idx = product_name.find(']')

  if start_idx != -1 and end_idx != -1:
      value = product_name[start_idx:end_idx+1]

      if '앱전용특가' in value:
        test_df['dcnt'][i] = 1
        new_product_name = product_name.replace(value, '')
        test_df['product_name'][i] = new_product_name

In [ ]:
# 깜짝특가

for i, product_name in enumerate(test_df['product_name']):

  start_idx = product_name.find('[')
  end_idx = product_name.find(']')

  if start_idx != -1 and end_idx != -1:
      value = product_name[start_idx:end_idx+1]

      if '깜짝특가' in value:
        test_df['dcnt'][i] = 1
        new_product_name = product_name.replace(value, '')
        test_df['product_name'][i] = new_product_name

In [ ]:
# 박스

for i, product_name in enumerate(test_df['product_name']):

  start_idx = product_name.find('[')
  end_idx = product_name.find(']')

  if start_idx != -1 and end_idx != -1:
      value = product_name[start_idx:end_idx+1]

      if 'BOX' in value:
        test_df['box'][i] = 1
        new_product_name = product_name.replace(value, '')
        test_df['product_name'][i] = new_product_name

In [ ]:
# 대괄호값 이용하여 중분류 채우기

for i, product_name in enumerate(test_df['product_name']):

  start_idx = product_name.find('[')
  end_idx = product_name.find(']')

  if start_idx != -1 and end_idx != -1:
      value = product_name[start_idx:end_idx+1]

      if '택배배송' in value:
        test_df['mid_cat2'][i] = '1001'
      if '쿡킷' in value:
        test_df['mid_cat2'][i] = '1002'

In [ ]:
# 제거 

for i, product_name in enumerate(test_df['product_name']):

  start_idx = product_name.find('[')
  end_idx = product_name.find(']')

  if start_idx != -1 and end_idx != -1:
      value = product_name[start_idx:end_idx+1]

      if value in event_list:
        new_product_name = product_name.replace(value, '')
        test_df['product_name'][i] = new_product_name

In [ ]:
event_list = []
product_list = test_df['product_name'].unique()

for product in product_list:

  start_idx = product.find('[')
  end_idx = product.find(']')

  # data.loc[data['product_name'] == product, 'product_name'] == re.search(r'\[(.*?)\]', product).group(1)
  
  if start_idx != -1 and end_idx != -1:
      extracted_value = product[start_idx:end_idx+1]
      event_list.append(extracted_value)

event_list = list(set(event_list))
event_list # 빈 리스트인지 확인!!!!!!!!!!!

['[냉동]', '[사은품]', '[쿡킷]', '[bibigo table]']

In [ ]:
# 한번 더 돌려주기 -> 대괄호값 이용하여 중분류 채우기

for i, product_name in enumerate(test_df['product_name']):

  start_idx = product_name.find('[')
  end_idx = product_name.find(']')

  if start_idx != -1 and end_idx != -1:
      value = product_name[start_idx:end_idx+1]

      if '택배배송' in value:
        test_df['mid_cat2'][i] = '1001'
      if '쿡킷' in value:
        test_df['mid_cat2'][i] = '1002'

In [ ]:
# 제거 

for i, product_name in enumerate(test_df['product_name']):

  start_idx = product_name.find('[')
  end_idx = product_name.find(']')

  if start_idx != -1 and end_idx != -1:
      value = product_name[start_idx:end_idx+1]

      if value in event_list:
        new_product_name = product_name.replace(value, '')
        test_df['product_name'][i] = new_product_name

In [ ]:
event_list = []
product_list = test_df['product_name'].unique()

for product in product_list:

  start_idx = product.find('[')
  end_idx = product.find(']')

  # data.loc[data['product_name'] == product, 'product_name'] == re.search(r'\[(.*?)\]', product).group(1)
  
  if start_idx != -1 and end_idx != -1:
      extracted_value = product[start_idx:end_idx+1]
      event_list.append(extracted_value)

event_list = list(set(event_list))
event_list # 빈 리스트인지 확인!!!!!!!!!!!

[]

## () 값 파생변수

In [ ]:
small_list = []
product_list = test_df['product_name'].unique()

for product in product_list:

  start_idx = product.find('(')
  end_idx = product.find(')')

  # data.loc[data['product_name'] == product, 'product_name'] == re.search(r'\[(.*?)\]', product).group(1)
  
  if start_idx != -1 and end_idx != -1:
      extracted_value = product[start_idx:end_idx+1]
      small_list.append(extracted_value)

small_list = list(set(small_list))
small_list

['(20g*20EA)',
 '(6개)',
 '(튜브)',
 '(1-2인분X5개입)',
 '(총600g)',
 '(1박스)',
 '(미국산,초이스등급)',
 '(100억)',
 '(20입x2묶음)',
 '(유통기한:2023-09-09)',
 '(중간입자)',
 '(4주)',
 '(꼬지)',
 '(용기형)',
 '(구이용)',
 '(1개월분)',
 '(간장숙성)',
 '(표고버섯 300g 감자 1kg 양파 1kg 애호박 1개)',
 '(미국산, 초이스등급)',
 '(얇은면)',
 '(20억)',
 '(한라봉, 천혜향, 레드향 각 4입)',
 '(넓은면)',
 '(유통기한 23.03.22)',
 '(60g*10입)',
 '(유통기한23.01.29까지)',
 '(50g*20입)',
 '(꽃게)',
 '(100g*7입)',
 '(10미)',
 '(총21토막)',
 '(사각면)',
 '(유통기한23.01.08까지)',
 '(플레인)',
 '(페퍼로니,고르곤졸라,콤비네이션,불고기)',
 '(노캡)',
 '(유통기한23.02.17까지)',
 '(45gx10봉)',
 '(용기)',
 '(1BOX)',
 '(중)',
 '(3~4입)',
 '(2인)',
 '(150g+150g)',
 '(유통기한:2023-04-23)',
 '(15일분)',
 '(컵)',
 '(6입x2묶음)',
 '(전복내장영양밥 200gX2개+불고기버섯영양밥 200gX2개+뿌리채소영양밥 200gX2개+꿀약밥 200gX2개+컵 코스터 2개입)',
 '(초이스 등급)',
 '(특상)',
 '(봉)',
 '(2개월)',
 '(냉동)',
 '(유통기한:2023-04-24)',
 '(유통기한:2023-04-25)',
 '(유통기한:2023-06-30)',
 '(27G*40개)',
 '(50gx25개)',
 '(1개월)',
 '(2인분)',
 '(3입)',
 '(스프레이오일세트)',
 '(유통기한:2023-05-22)',
 '(와사비맛)',
 '(딸기, 블루베리, 초코볼)',
 '(윙)',
 '(감바스,나시고랭,불고기,치

In [ ]:
test_df['expir'] = 0

In [ ]:
# 유통기한 임박 제품(한번 더 해줘야 함)
for i, product_name in enumerate(test_df['product_name']):

  start_idx = product_name.find('(')
  end_idx = product_name.find(')')

  if start_idx != -1 and end_idx != -1:
      value = product_name[start_idx:end_idx+1]

      if '유통기한' in value:
        test_df['expir'][i] = 1
       # new_product_name = product_name.replace(value, '')
       # test_df['product_name'][i] = new_product_name

In [ ]:
for i, product_name in enumerate(test_df['product_name']):

  start_idx = product_name.find('(')
  end_idx = product_name.find(')')

  if start_idx != -1 and end_idx != -1:
      value = product_name[start_idx:end_idx+1]

      if ('box' in value) | ('BOX' in value) | ('박스' in value):
        test_df['box'][i] = 1
       # new_product_name = product_name.replace(value, '')
       # test_df['product_name'][i] = new_product_name

In [ ]:
# # 제거

# for i, product_name in enumerate(df['product_name']):

#  start_idx = product_name.find('(')
#  end_idx = product_name.find(')')

#  if start_idx != -1 and end_idx != -1:
#      value = product_name[start_idx:end_idx+1]

#      if value in small_list:
#        new_product_name = product_name.replace(value, '')
#        df['product_name'][i] = new_product_name

## 세트 상품

In [ ]:
#for i in range(len(test_df)):
#  if '+' in test_df['product_name'][i]:
#    print(test_df['product_name'][i])
#    print(test_df['new_product_name'][i])
#    print()

In [ ]:
set_list = []

for i in range(len(test_df)):
  if '+' in test_df['product_name'][i]:
    set_list.append(test_df['product_name'][i].strip())

set_list = list(set(set_list))

In [ ]:
substring_list = ['리턴업', '루테인']

# Use list comprehension to filter out values that contain the substring
for substring in substring_list:
  filtered_list = [value for value in set_list if substring not in value]

filtered_list

['리턴업 갱년기케어 회화나무열매추출물+비타민Dx2개',
 '(냉동) 비비고 테이블 특설렁탕 700x3개+(냉동) 비비고 테이블 특양지곰탕 700gx3개',
 '햇반쿡반 참치마요주먹밥 500gx2개+햇반쿡반 스팸김치주먹밥 500gx2개',
 '고메 트러플 크림파스타 740g+고메 머쉬룸 투움바 파스타 740g+고메 봉골레 오일파스타 626g',
 '모닝두부140gX6개 + 오곡참깨드레싱 105g',
 '양갈비 프렌치랙 300g+시즈닝 2종',
 '고메 페퍼로니토마토 시카고피자 390g+고메 클래식 콤비네이션 피자 405g+고메 클래식 불고기 피자 405g',
 '밸런스밀 귀리 50gX4개+밸런스밀 견과 50gX4개+밸런스밀 고구마 50gX4개+밸런스밀 바나나 50gX4개+밸런스밀 프로틴 카카오 50gX4개',
 '양갈비 어깨살 300g+시즈닝 2종',
 '양갈비 숄더랙 300g+시즈닝 2종',
 '비비고 왕교자 1.05kgX2개+(냉동) 비비고 테이블 특양지곰탕 700gX2개',
 '밸런스밀 귀리 50gx2개+밸런스밀 견과 50gx2개+밸런스밀 고구마 50gx2개+밸런스밀 바나나 50gx2개+밸런스밀 프로틴쉐이크 카카오 50gx2개',
 '가쓰오우동 2인분+튀김우동 2인분',
 '비비고왕교자1.05kgX2개+고메 바삭튀겨낸 통등심돈카츠450gX2개',
 '가쓰오우동 2인분+튀김우동 2인분X2개',
 '햇반 쿡반 간장버터장조림볶음밥 400gx2개+갓츄 핫소스 샘플키트X2개',
 '더건강한 닭가슴살 직화 통살구이100gX3개+직화 스테이크100gX3개+소시지120gX3개',
 '더건강한 닭가슴살 직화 통살구이100gx2개+갓츄 핫소스 샘플키트',
 '비비고 왕교자 1.05kgX3개+비비고 왕교자 라이트 455g*2EAX3개',
 '(냉동) 비비고 테이블 특 선물세트 (특양지곰탕700gx2개+특설렁탕700gx1개)',
 '더건강한 닭가슴살 직화 화다닭 75gX3개+더건강한 닭가슴살 쏙쏙한입큐브80gX3개+더건강한 닭가슴살 비엔나 120gX3개',
 '백설 리얼 배도

In [ ]:
# 세트 상품 파생변수

test_df['set'] = 0
for i in range(len(test_df)):
  if test_df['product_name'][i] in filtered_list:
    test_df['set'][i] = 1

In [ ]:
test_df[test_df['set'] == 1]

,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,employee_yn,order_date,prime_yn,total_order_amt,...,pre_dcnt,colab,new_years,early_del,exclusive,dcnt,box,mid_cat2,expir,set
36,20230101965937,꼬마 돈까스 400gX4개+꼬마너겟 320gX4개,1,10.041640,M,4,N,20230101,NaN,111.665878,...,0,0,0,0,0,0,0,0,0,1
55,20230101969192,찹쌀 호떡 믹스 400gX2개+백설 초당옥수수호떡믹스 300g+백설 유러피안 해바라...,1,9.602518,M,4,Y,20230101,NaN,92.178690,...,0,0,0,0,0,0,0,0,0,1
77,20230101964922,비비고 왕교자 1.05kgX2개+진한고기만두 400g*2번들X2개+김치왕교자 420...,1,10.774111,F,4,N,20230101,NaN,28.796397,...,0,0,0,0,0,0,0,0,0,1
161,20230101968407,찹쌀 호떡 믹스 400gX2개+백설 초당옥수수호떡믹스 300g+백설 유러피안 해바라...,1,9.585965,M,4,N,20230101,NaN,9.585965,...,0,0,0,0,0,0,0,0,0,1
164,20230101965163,밸런스밀 귀리 50gx2개+밸런스밀 견과 50gx2개+밸런스밀 고구마 50gx2개+...,1,9.718362,M,5,Y,20230101,NaN,34.933378,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19505,20230113095516,비비고왕교자1.05kg+남도떡갈비+한입떡갈비+도톰동그랑땡+잔칫집모둠잡채+백설 너비아니,1,10.341291,M,3,Y,20230113,NaN,10.341291,...,0,0,0,0,0,0,0,0,0,1
19508,20230113093965,햇반솥반 미미달 기획세트(전복내장영양밥 200gX2개+불고기버섯영양밥 200gX2개...,1,9.891415,M,5,Y,20230113,NaN,9.891415,...,0,0,0,0,0,0,0,0,0,1
19604,20230111076388,밸런스밀 귀리 50gx2개+밸런스밀 견과 50gx2개+밸런스밀 고구마 50gx2개+...,1,9.718362,F,3,Y,20230111,NaN,9.718362,...,0,0,0,0,0,1,0,0,0,1
19610,20230115106090,밸런스밀 귀리 50gx2개+밸런스밀 견과 50gx2개+밸런스밀 고구마 50gx2개+...,1,9.718362,M,5,Y,20230115,NaN,9.718362,...,0,0,0,0,0,1,0,0,0,1


## _오특, _행사, _패밀리데이 파생변수

In [ ]:
test_df['event'] = 0

for i, product_name in enumerate(test_df['product_name']):
  if '_오특' in product_name:
    test_df['dcnt'][i] = 1
  if ('_행사' in product_name) or ('패밀리데이' in product_name):
    test_df['event'][i] = 1

In [ ]:
## []까지 제거한거: product_name
## ()와 정규식 모두 제거한거: prod_processed

test_df['prod_processed'] = test_df['product_name']

## 정규식 제거

In [ ]:
test_df.head()

,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,employee_yn,order_date,prime_yn,total_order_amt,...,new_years,early_del,exclusive,dcnt,box,mid_cat2,expir,set,event,prod_processed
0,20230101964282,비비고 스팸부대찌개 460g,1,8.161946,M,4,N,20230101,NaN,32.658027,...,0,0,0,0,0,0,0,0,0,비비고 스팸부대찌개 460g
1,20230101964282,비비고 두부김치찌개 460g,1,7.920083,M,4,N,20230101,NaN,32.658027,...,0,0,0,0,0,0,0,0,0,비비고 두부김치찌개 460g
2,20230101964282,비비고 차돌된장찌개 460g,1,8.161946,M,4,N,20230101,NaN,32.658027,...,0,0,0,0,0,0,0,0,0,비비고 차돌된장찌개 460g
3,20230101964282,비비고 수삼갈비탕 400g,1,8.414052,M,4,N,20230101,NaN,32.658027,...,0,0,0,0,0,0,0,0,0,비비고 수삼갈비탕 400g
4,20230101970142,삼호 생선살어묵 야채 200g,1,8.098947,M,4,Y,20230101,NaN,52.732733,...,0,0,0,0,0,0,0,0,0,삼호 생선살어묵 야채 200g


In [ ]:
test_df['festival'].value_counts()

1    10322
0     9338
Name: festival, dtype: int64

In [ ]:
prod_name = test_df.loc[:,'product_name']

def remove_square_brackets(text):
    result = re.sub(r'\[.*?\]', '', text)
    return result.strip()


# 리스트의 모든 요소에 대해 괄호와 그 안의 문자를 삭제
prod_name = [remove_square_brackets(item) for item in prod_name]
len(prod_name)


def extract_patterns(lst):
    patterns = []
    for text in lst:
        pattern = re.search(r'[A-Za-z가-힣\s]+', text)
        if pattern:
            patterns.append(pattern.group().strip())
        else:
            patterns.append('')
    return patterns


refined_prod_name = extract_patterns(prod_name)

test_df['prod_processed'] = refined_prod_name

# middle_cat(중분류) 전처리

product_name: [] 제거된거
 
prod_processed: 모두 다 제거된거 

In [ ]:
mid.head()

,product_name,middle,prod_processed
0,해찬들 사계절쌈장 500g,4010,해찬들 사계절쌈장
1,해찬들 새콤달콤 초고추장 1050g,4010,해찬들 새콤달콤 초고추장
2,해찬들 고기찍어먹는장 찍장 쌈장 300g,4010,해찬들 고기찍어먹는장 찍장 쌈장
3,해찬들 궁중식 쇠고기 볶음고추장 500g,4010,해찬들 궁중식 쇠고기 볶음고추장
4,해찬들 100% 우리쌀 태양초 고추장 1kg,4010,해찬들


In [ ]:
#브랜드 사전 
middledic = dict(zip(mid['prod_processed'], mid['middle']))

#브랜드명 부여
test_df['middle_cat'] = test_df['prod_processed'].map(middledic).fillna(np.nan)

### 1. 전처리 안된 값이랑 비교하여 중분류 넣기
mid_id_list = ['0004010', '0004011', '0004012', '0004013', '0004014', '0004015', '000407', '000408', '000409', \
               '000501', '000502', '000503', '000505', '000601', '000602', '000603', '000604', '000702', '000703', '000705',\
               '000901', '000902', '000903', '000904', '000905', '001001', '001002', '001003', \
               '001801', '001802', '001803', '001804', '001805', '001806', '001807', '001901', '001903', '001904', '001905', '001906', '001907', \
               '002001', '002002', '002003', '002004', '002005', '002101', '002102', '002104', '002105', '002201', '002202', '002203', \
               '002301', '002302', '002303', '002304']

mid_data = {} 

%cd /content/drive/MyDrive/Colab Notebooks/BDA 공모전/BDA 공모전/중분류
for id in mid_id_list:
  with open(id + '.txt', 'r', encoding='cp949') as file:
    lines = [line.strip() for line in file]
    unique_lines = list(set(lines)) # 중복값 제거
    mid_data[id] = unique_lines

test_df['mid_cat2'] = test_df['product_name'].apply(lambda x: next((id for id in mid_id_list if any(item in x for item in mid_data[id])), 0))


### 2. 전처리한 중분류값이랑 비교하여 값 채우기

test_df['middle_cat'] = test_df['middle_cat'].fillna(0).astype(int) # 대분류 값 정수형으로 변환
test_df['mid_cat2'] = test_df['mid_cat2'].astype(int)



#브랜드 사전
middledic = dict(zip(mid['prod_processed'], mid['middle']))

#대분류명 부여
for i in range(len(test_df)):
  if test_df['mid_cat2'][i] == 0:
    test_df['mid_cat2'][i] = test_df['middle_cat'][i]

test_df[test_df['mid_cat2'] == 0]['prod_processed'].value_counts()


### 3. 은미가 전처리한거 반영
#%cd /content/drive/MyDrive/Colab Notebooks/BDA 공모전/BDA 공모전/이거다
#eunmi = pd.read_csv('thisisdata.csv')
eunmi = pd.read_csv('/content/drive/MyDrive/BDA 공모전/이거다/thisisdata.csv')
eunmi = eunmi.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1)

for i in range(len(test_df)):
  if test_df['mid_cat2'][i] == 0:
    test_df['mid_cat2'][i] = eunmi['middle_cat2'][i]


### 4. 노가다
search_dic = {'빅반들':1802, '크레잇 콰트로 콘맥앤치즈':2102, '숯불향고추장삼겹살':904, '삼호 가쓰오 모둠 어묵탕':1906, '비비고 순살고등어구이':1904, \
              '비비고 언양식바싹불고기':2203, '미초':501, '얼큰우동':1805, 'VIPS':1003, '유산균':601, '이너비':604, '쁘띠첼':2104, '특별한선택':2301, '유러피안':414, '통삼겹스테이크':2302, '리턴업':603, '드레싱':407, '행복한콩':1906, \
                   '사골순댓국':1901, '참기름':414, '스팸고급유':2301, '핫도그':2101, '죽':1804, '소스':407, '국수':1805, '닭가슴살 통살':2304, \
                    '컵반': 1802, 'CJ명가':1907, '빕스':1003, '흑삼':602, '최고의선택':2301, '스팸스위트':2301, '쿡킷':1002, 'BIG BEST':1802, '믹스':409, \
                    '화다닭':2304, '스팸듬뿍부대찌개':1002, '스팸':2301, 'Black Label':2301, '올리브유':414, 'EXCYCLE':2105, '식혜':502, '스모크 치킨':407, \
                     'BYO':601, '맛밤':2104, '맥스봉':2101, '빅반들':1802, '크레잇 콰트로 콘맥앤치즈':2102, '숯불향고추장삼겹살':904, '삼호 가쓰오 모둠 어묵탕':1906, \
              '순살고등어구이':1904, '언양식바싹불고기':2203, '비엔나':2303, '스파게티':1805, '아이시스':502, '백설 프리미엄':414, '칠성사이다':502, \
              '그레인보울':1807,'다담':408, '닭가슴살':2304, '햄스빌 베이컨':2302, '베이컨':2302, '된장찌개양념':410, '된장찌개 양념':410, '만두':2001, \
                '그릴후랑크':2303, '투썸':505, '한우':703, '쌈장':410, '숯불향 치즈닭갈비':1002, '한뿌리':602, '쿠키':2102, '생선조림양념':408, \
                '직화구이 백설너비아니':2203, '씨푸드':407, '백설':414, '삼호':1906, '찹스테이크':1002, '두부':1906, '밸런스밀':1807, 'BIG TRAY':1802, \
                '숯불향':1002, '도시락':1807, '트레비':502, '계절밥상':1003, '감바스':1002, '채끝 구이용':703, '이츠웰': 702, '프레시웨이 LA갈비':703, '프레시웨이':702, \
                '정육':703, '쌈무':1905, '하선정':412, '호주산':703, '오리훈제':703, '포도씨유':414, '동물복지':1906, \
                '샐러드':702,'두부면':1906,'면':1805, '국':1901, '솜사탕':2102, '그라탕':2001, '해신탕':1001, '김치':1903, '유정란':1905, \
                '치킨':2003,'너겟':2003, '닭한마리':1001, '초반': 2004, '커피':505, '돈까스':2201, '양고기 선물세트':703, '쯔란':1002, '고기':1002, \
                '김말이':2102, '고등어':1904, '아이누리':502, '삼천포':1002, '파스타':1805, '고추장':410, 'BLACKLABEL':2301, '천혜향':702, \
                '샤인머스캣':702, '레드향':702, '자몽':702, '양배추':602, '배':702, '골드키위':702, '체리':702, '한정SET':2004, '간짜장':2004, '시럽':503, \
                '서울우유': 2105, '불낙전골':1002, '버섯':702, '정품진':703, '살치살':703, '찜갈비':703, '견과':1807, '제주':705, '굴비':705, \
                '한과':2102, '들깨':1002, '부채살스테이크':1002, '살':703, '보일링랍스터':1001, '프리미엄쌀':702, '치즈파우더':407, '달고나':2102, \
                '멸치':705, '겉절이':1002, '평양식 어복쟁반':1002, '새우까스':2201, '튀김':2201, '흑화고세트':702, '건강쌀':702, '몽골리안비프':1002, \
                  '풀드포크':2302, '아보카도':702,'깐쇼새우':2004,'망고선물세트':702}

keys = search_dic.keys()

for key in keys:
  for i in range(len(test_df)):
    if test_df['mid_cat2'][i] == 0:
      if key in test_df['prod_processed'][i]:
        # print(test_df['product_name'][i] + '////' + test_df['prod_processed'][i])
        test_df['mid_cat2'][i] = search_dic[key]


### 마지막 처리

for i in range(len(test_df)):
  if test_df['mid_cat2'][i] == 4010:
    test_df['mid_cat2'][i] = 410
  elif test_df['mid_cat2'][i] == 4011:
    test_df['mid_cat2'][i] = 411
  elif test_df['mid_cat2'][i] == 4012:
    test_df['mid_cat2'][i] = 412
  elif test_df['mid_cat2'][i] == 4013:
    test_df['mid_cat2'][i] = 413
  elif test_df['mid_cat2'][i] == 4014:
    test_df['mid_cat2'][i] = 414
  elif test_df['mid_cat2'][i] == 4015:
    test_df['mid_cat2'][i] = 415
  else:
    continue

[Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/BDA 공모전/BDA 공모전/중분류'
/content




---



# main_cat (대분류)

In [ ]:
test_df['main_cat2'] = test_df['mid_cat2'] // 100
test_df['main_cat2'].unique()

array([19, 18, 23, 20, 22, 21,  4,  6, 10,  9,  7,  0,  5])

In [ ]:
test_df.drop(['middle_cat'], axis = 1, inplace = True)

In [ ]:
test_df.head(3)

,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,employee_yn,order_date,prime_yn,total_order_amt,...,early_del,exclusive,dcnt,box,mid_cat2,expir,set,event,prod_processed,main_cat2
0,20230101964282,비비고 스팸부대찌개 460g,1,8.161946,M,4,N,20230101,NaN,32.658027,...,0,0,0,0,1901,0,0,0,비비고 스팸부대찌개,19
1,20230101964282,비비고 두부김치찌개 460g,1,7.920083,M,4,N,20230101,NaN,32.658027,...,0,0,0,0,1901,0,0,0,비비고 두부김치찌개,19
2,20230101964282,비비고 차돌된장찌개 460g,1,8.161946,M,4,N,20230101,NaN,32.658027,...,0,0,0,0,1901,0,0,0,비비고 차돌된장찌개,19


# weight, capacity 

In [ ]:
# 정규식 패턴
pattern = r'(\d+(?:\.\d+)?(?:kg|mL|mg|ml|g|MG|G|ML|L|KG|Kg))'
# 정규식을 이용하여 매칭되는 데이터 추출
test_df['matches'] = test_df['product_name'].str.extractall(pattern).groupby(level=0).first()[0]
# 문자열 분할 및 추출
weight = test_df.loc[:,['product_name','matches']]



def process_matches(value):
    match = re.match(r'(\d+(\.\d+)?)([a-zA-Z]+)', value)
    if match:
        num = match.group(1)
        unit = match.group(3)
        if unit in ('mg', 'MG'):
            return 0
        elif unit in ('g','G', 'mL','ml','ML'):
            return num
        elif unit in ('KG', 'Kg', 'kg', 'L'):
            return float(num) * 1000
    return 0


weight['weight'] = weight['matches'].apply(str).apply(process_matches)
weight['weight'] = weight['weight'].astype(float)



weight['capacity'] = 0 


weight.loc[weight['weight'] >= 1000, 'capacity'] = 1  # 조건을 만족하는 경우 1 할당


kilogram = weight[['product_name','weight','capacity']]

In [ ]:
kilogram.isna().sum()

product_name    0
weight          0
capacity        0
dtype: int64

In [ ]:
kdic = dict(zip(kilogram['product_name'], kilogram['capacity']))

#브랜드사전에 있는 제품은 그에 맞는 '브랜드'명을 저장 (없으면 nan)
test_df['capacity'] = test_df['product_name'].map(kdic).fillna(np.nan)

In [ ]:
wdic = dict(zip(kilogram['product_name'], kilogram['weight']))

#브랜드사전에 있는 제품은 그에 맞는 '브랜드'명을 저장 (없으면 nan)
test_df['weight'] = test_df['product_name'].map(wdic).fillna(np.nan)

In [ ]:
test_df.head(3)

,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,employee_yn,order_date,prime_yn,age_gen,...,box,mid_cat2,expir,set,event,prod_processed,main_cat2,matches,capacity,weight
0,2.023010e+13,비비고 스팸부대찌개 460g,1.0,8.161946,M,4.0,N,20230101,NaN,4050 M,...,0,1901,0,0,0,비비고 스팸부대찌개,19,460g,0,460.0
1,2.023010e+13,비비고 두부김치찌개 460g,1.0,7.920083,M,4.0,N,20230101,NaN,4050 M,...,0,1901,0,0,0,비비고 두부김치찌개,19,460g,0,460.0
2,2.023010e+13,비비고 차돌된장찌개 460g,1.0,8.161946,M,4.0,N,20230101,NaN,4050 M,...,0,1901,0,0,0,비비고 차돌된장찌개,19,460g,0,460.0


# 일반회원과 임직원 분리하기

In [ ]:
emp_df = test_df[test_df['employee_yn'] == 'Y'].reset_index(drop=True)
reg_df = test_df[test_df['employee_yn'] == 'N'].reset_index(drop=True)

# 임직원 선물하기 변수

In [ ]:
#임직원인 경우에만 해당됨 코드가 넘 오래걸려서 맨 마지막 변수로 추가하자 
#일곱번째 변수: 선물하기 유무 

 #선물하기 카테고리 크롤링
url = 'https://www.cjthemarket.com/pc/gift/giftProd?wid1=lnb_gift'

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36'}
html = requests.get(url, headers = headers)
soup = BeautifulSoup(html.text, "html.parser")

gift_items = soup.find_all('p', class_='product-title')

gift_item_list = []


for item in gift_items:
  item_name = item.text
  item_name = item_name.strip()
  gift_item_list.append(item_name)


emp_df['gift'] = 0
for i in range(len(emp_df)):
  if emp_df['product_name'][i] in gift_item_list:
    for j in range(len(emp_df)): # 해당 주문번호에 해당하는 row 모두 gift == 1
      if emp_df['scd'][j] == emp_df['scd'][i]:
        emp_df['gift'][j] = 1

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/BDA 공모전/BDA 공모전
import os
os.getcwd()

/content/drive/.shortcut-targets-by-id/11I-rKkyrnUaOhSTzVScvbROijR4qxGDQ/BDA 공모전


'/content/drive/.shortcut-targets-by-id/11I-rKkyrnUaOhSTzVScvbROijR4qxGDQ/BDA 공모전'

In [ ]:
reg_df.to_csv('reg_test.csv')
emp_df.to_csv('emp_test.csv')

In [ ]:
reg_df

,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,employee_yn,order_date,prime_yn,total_order_amt,...,box,mid_cat2,expir,set,event,prod_processed,main_cat2,matches,capacity,weight
0,20230101964282,비비고 스팸부대찌개 460g,1,8.161946,M,4,N,20230101,NaN,32.658027,...,0,1901,0,0,0,비비고 스팸부대찌개,19,460g,0,460.0
1,20230101964282,비비고 두부김치찌개 460g,1,7.920083,M,4,N,20230101,NaN,32.658027,...,0,1901,0,0,0,비비고 두부김치찌개,19,460g,0,460.0
2,20230101964282,비비고 차돌된장찌개 460g,1,8.161946,M,4,N,20230101,NaN,32.658027,...,0,1901,0,0,0,비비고 차돌된장찌개,19,460g,0,460.0
3,20230101964282,비비고 수삼갈비탕 400g,1,8.414052,M,4,N,20230101,NaN,32.658027,...,0,1901,0,0,0,비비고 수삼갈비탕,19,400g,0,400.0
4,20230101965237,크레잇 블랙페퍼 폭찹 스테이크,1,8.277412,F,4,N,20230101,NaN,83.641997,...,0,2304,0,0,0,크레잇 블랙페퍼 폭찹 스테이크,23,NaN,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11609,20230103990299,햇반 쿡반 참치마요주먹밥 500g,1,8.414052,F,5,N,20230103,NaN,8.414052,...,0,1803,0,0,0,햇반 쿡반 참치마요주먹밥,18,500g,0,500.0
11610,20230103994754,얼큰 버섯 비비고 만두 전골_밀키트,1,9.112066,F,4,N,20230103,NaN,9.112066,...,0,1002,0,0,0,얼큰 버섯 비비고 만두 전골,10,NaN,0,0.0
11611,20230103989403,비비고 왕교자 455gx2개,2,9.215427,M,2,N,20230103,NaN,18.430855,...,0,2001,0,0,0,비비고 왕교자,20,455g,0,455.0
11612,20230103990277,비비고 왕교자 455gx2개,1,8.917177,F,3,N,20230103,NaN,8.917177,...,0,2001,0,0,0,비비고 왕교자,20,455g,0,455.0


In [ ]:
emp_df

,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,employee_yn,order_date,prime_yn,total_order_amt,...,mid_cat2,expir,set,event,prod_processed,main_cat2,matches,capacity,weight,gift
0,20230101970142,삼호 생선살어묵 야채 200g,1,8.098947,M,4,Y,20230101,NaN,52.732733,...,1801,0,0,0,삼호 생선살어묵 야채,18,200g,0,200.0,0
1,20230101970142,삼호 가쓰오모둠어묵탕413g,1,8.993303,M,4,Y,20230101,NaN,52.732733,...,1906,0,0,0,삼호 가쓰오모둠어묵탕,19,413g,0,413.0,0
2,20230101970142,(냉동) 비비고 테이블 특양지곰탕 700g,1,9.177714,M,4,Y,20230101,NaN,52.732733,...,1901,0,0,0,냉동,19,700g,0,700.0,0
3,20230101970142,백설 햄스빌두툼한굿베이컨340gX2개,1,9.309552,M,4,Y,20230101,NaN,52.732733,...,2302,0,0,0,백설 햄스빌두툼한굿베이컨,23,340g,0,340.0,0
4,20230101970142,삼호 떡보도반한찹쌀어묵 350g,1,8.169903,M,4,Y,20230101,NaN,52.732733,...,2004,0,0,0,삼호 떡보도반한찹쌀어묵,20,350g,0,350.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8041,20230115105218,CJ명가 토종김 3S호X5,2,11.953043,M,4,Y,20230115,NaN,23.906086,...,2001,0,0,0,CJ명가 토종김,20,NaN,0,0.0,0
8042,20230103992088,청도 반건시 선물세트 1.5kg,6,12.476867,M,3,Y,20230103,NaN,74.861200,...,702,0,0,0,청도 반건시 선물세트,7,1.5kg,1,1500.0,0
8043,20230103990367,프레시웨이 LA갈비 선물세트 3.2kg,1,12.204807,M,3,Y,20230103,NaN,12.204807,...,1901,0,0,0,프레시웨이 LA갈비 선물세트,19,3.2kg,1,3200.0,0
8044,20230103991606,비비고 왕교자 455gx2개,4,10.437903,M,3,Y,20230103,NaN,41.751611,...,2001,0,0,0,비비고 왕교자,20,455g,0,455.0,0


In [ ]:
#reg_df.to_csv('reg_train.csv')

In [ ]:
emp_df

,scd,product_name,net_order_qty,net_order_amt,gender,age_grp,employee_yn,order_date,prime_yn,age_gen,...,mid_cat2,expir,set,event,prod_processed,main_cat2,matches,capacity,weight,gift
0,2.023010e+13,삼호 생선살어묵 야채 200g,1.0,8.098947,M,4.0,Y,20230101,NaN,4050 M,...,1801,0,0,0,삼호 생선살어묵 야채,18,200g,0,200.0,0
1,2.023010e+13,삼호 가쓰오모둠어묵탕413g,1.0,8.993303,M,4.0,Y,20230101,NaN,4050 M,...,1906,0,0,0,삼호 가쓰오모둠어묵탕,19,413g,0,413.0,0
2,2.023010e+13,(냉동) 비비고 테이블 특양지곰탕 700g,1.0,9.177714,M,4.0,Y,20230101,NaN,4050 M,...,1901,0,0,0,냉동,19,700g,0,700.0,0
3,2.023010e+13,백설 햄스빌두툼한굿베이컨340gX2개,1.0,9.309552,M,4.0,Y,20230101,NaN,4050 M,...,2302,0,0,0,백설 햄스빌두툼한굿베이컨,23,340g,0,340.0,0
4,2.023010e+13,삼호 떡보도반한찹쌀어묵 350g,1.0,8.169903,M,4.0,Y,20230101,NaN,4050 M,...,2004,0,0,0,삼호 떡보도반한찹쌀어묵,20,350g,0,350.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8041,2.023012e+13,CJ명가 토종김 3S호X5,2.0,11.953043,M,4.0,Y,20230115,NaN,4050 M,...,2001,0,0,0,CJ명가 토종김,20,NaN,0,0.0,0
8042,2.023010e+13,청도 반건시 선물세트 1.5kg,6.0,12.476867,M,3.0,Y,20230103,NaN,2030 M,...,702,0,0,0,청도 반건시 선물세트,7,1.5kg,1,1500.0,0
8043,2.023010e+13,프레시웨이 LA갈비 선물세트 3.2kg,1.0,12.204807,M,3.0,Y,20230103,NaN,2030 M,...,1901,0,0,0,프레시웨이 LA갈비 선물세트,19,3.2kg,1,3200.0,0
8044,2.023010e+13,비비고 왕교자 455gx2개,4.0,10.437903,M,3.0,Y,20230103,NaN,2030 M,...,2001,0,0,0,비비고 왕교자,20,455g,0,455.0,0
